In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import integrate

In [ ]:
from functools import reduce
from operator import matmul
from typing import Callable


def create_char_pol(*roots: complex) -> Callable[[np.ndarray], np.ndarray]:
    """
    Create characteristic polynomial from roots dynamically
    Return function that look like x -> (x - root[0]) @ (x - root[1]) @ ...
    (Subtraction is not element-wise, it's like in true math!)
    """
    dim = len(roots)

    char_pol = lambda x: (
        reduce(matmul, [x - np.identity(dim) * root for root in roots])
    )

    return char_pol

In [ ]:
class PendODESystem:
    def __init__(
            self,
            pend_mass=0.127,
            cart_mass=0.12*4,
            moment_of_inertia=7.631e-4,
            length=0.066,
            K_f=1.726,
            K_s=4.487,
            B_c=0,
            B_p=0.002,
    ) -> None:

        A_0 = np.array([
            [pend_mass + cart_mass, -pend_mass * length],
            [-pend_mass * length, moment_of_inertia + pend_mass * length * length]
        ])

        A_1 = np.array([
            [B_c, 0],
            [0, B_p]
        ])

        A_2 = np.array([
            [0, 0],
            [0, -pend_mass * 9.8 * length]
        ])

        first = -np.linalg.inv(A_0) @ A_2
        second = -np.linalg.inv(A_0) @ A_1
        third = np.linalg.inv(A_0) @ np.array([[1], [0]])

        self._A = np.array([
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [first[0][0], first[0][1], second[0][0], second[0][1]],
            [first[1][0], first[1][1], second[1][0], second[1][1]]
        ], dtype=np.complex_)

        self._b = np.array([
            [0],
            [0],
            third[0],
            third[1]
        ], dtype=np.complex_)

    @property
    def A(self):
        return self._A

    @property
    def b(self):
        return self._b

In [ ]:
pend = PendODESystem()

In [ ]:
C = np.column_stack([
    pend.b,
    pend.A @ pend.b,
    np.linalg.matrix_power(pend.A, 2) @ pend.b,
    np.linalg.matrix_power(pend.A, 3) @ pend.b,
])

print(C)
print(C.shape)
print(f"Rank: {np.linalg.matrix_rank(C)}")

In [ ]:
print(f"Eigs: {np.linalg.eigvals(pend.A)}")

In [ ]:
theta = (
    -np.array([[0, 0, 0, 1]])
    @ np.linalg.inv(C)
    @ create_char_pol(-2, complex(-1, -1), complex(-1, 1), -9)(pend.A)
)

print(f"{theta=}")


In [ ]:
np.linalg.eigvals(pend.A + pend.b @ theta)

In [ ]:
def linear_system(
        t: np.ndarray,
        x: np.ndarray,
        A: np.ndarray,
        b: np.ndarray,
        theta: np.ndarray,
        x_0: np.ndarray | None = None
) -> np.ndarray:
    if x_0 is None:
        x_0 = x

    return A @ x + b @ theta @ x_0

In [ ]:
start, stop = 0, 10

time = np.linspace(start, stop, 300)
y_0 = np.array([0, 0.1, 0, 0])

sol = integrate.solve_ivp(
    linear_system,
    (start, stop),
    y_0,
    dense_output=True,
    args=(pend.A, pend.b, theta),
    method="RK45"
)

In [ ]:
z = sol.sol(time)

In [ ]:
y_labels = (r"x", r"\phi", r"\dot x", r"\dot \phi")
# plt.rcParams['text.usetex'] = True # uncomment if you have latex

fig, axs = plt.subplots(4, 1)
fig.set_size_inches(10, 15)

for i in range(4):
    axs[i].plot(time, z[i], label="linear")
    axs[i].set_xlabel('time')
    axs[i].set_ylabel(y_labels[i])
    axs[i].grid(True)
    axs[i].legend()

fig.tight_layout()
# fig.savefig('out.png', dpi=300, facecolor='white') # uncomment to save high-res picture
plt.show()
